In [4]:
print("Importing...")
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import numpy as np
import torch
import torch.nn.functional as F
from matplotlib import pyplot as plt
from GCN import *
from Utils.math_distances import cosine_distance
from Utils.my_utils import *
from Utils.util import *
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter

import time
from train_utils import get_batch_data, evaluate
from train_utils import *


torch.manual_seed(124)
np.random.seed(124)

print("Finished Importing")

Importing...
Finished Importing


In [2]:
torch.manual_seed(124)
np.random.seed(124)

print("Finished Importing")

print("Settings")

run_folder="../"
dataset = "MeviyData"
STEP_dataset = "D:/3DStepGraphClassification_RapidDesignData/Datasets/RapidPrototype_TestData/StepData/"
graphml_dataset = "D:/3DStepGraphClassification_RapidDesignData/Datasets/RapidPrototype_TestData/GraphData/"
learning_rate=0.0005
batch_size=1
num_epochs=1
dropout=0.5
model_name = "GCN_model_02-16" # "Name of the model trained in train files"
model_path = "D:/3DStepGraphClassification_RapidDesignData/Datasets/RapidPrototype_TestData/GCN_model_02-16_100/Models/" + model_name

print("Using model at path:", model_path)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("The calculations will be performed on the device:", device)

# save paths
out_dir = "D:/3DStepGraphClassification_RapidDesignData/Output"
print("Results will be saved in:", out_dir)


Finished Importing
Settings
Using model at path: D:/3DStepGraphClassification_RapidDesignData/Datasets/RapidPrototype_TestData/GCN_model_02-16_100/Models/GCN_model_02-16
The calculations will be performed on the device: cuda:0
Results will be saved in: D:/3DStepGraphClassification/Output


In [6]:
print("Loading Graph data...")
use_degree_as_tag = False
fold = 0
# graphs, num_classes, filenames = my_load_data_with_filename(graphml_dataset, use_degree_as_tag)
graphs, num_classes = my_load_data_without_graph_load(graphml_dataset)
filenames = extract_filenames(graphs)

train_graphs, test_graphs = separate_data_new(graphs, split_size=0.1)
train_graphs, valid_graphs = separate_data_new(train_graphs, split_size=0.1)
print("# training graphs: ", len(train_graphs))
# print_data_commposition(train_graphs)
print("# validation graphs: ", len(valid_graphs))
# print_data_commposition(valid_graphs)
print("# test graphs: ", len(test_graphs))
# print_data_commposition(test_graphs)

# feature_dim_size = graphs[0].node_features.shape[1]
feature_dim_size = 64
print("Loading data... finished!")

Loading Graph data...
# training graphs:  405
# validation graphs:  45
# test graphs:  50
Loading data... finished!


In [7]:
print("Creating model")

num_classes = 393
# model = GCN_CN_v4(feature_dim_size=feature_dim_size, num_classes=num_classes, dropout=dropout).to(device)
model = GCN_CN_v4(feature_dim_size=feature_dim_size, num_classes=num_classes, dropout=dropout).to(device)
model.load_state_dict(torch.load(model_path))
children_counter = 0
for n,c in model.named_children():
    print("Children Counter: ",children_counter," Layer Name: ",n,)
    children_counter+=1
output_layer = "attention"

class feature_extractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained = model
        self.pretrained.eval()

        self.net = list(self.pretrained.children())[:]#-2
        self.pretrained = None

    def forward(self, adj, features):
        features = self.net[0](x=features, edge_index=adj)
        features = nn.functional.relu(features)
        features = self.net[1](x=features, edge_index=adj)
        features = nn.functional.relu(features)
        features = self.net[2](x=features, edge_index=adj)
        features = nn.functional.relu(features)
        scores = self.net[3](features)
        scores = torch.t(scores)

        scores = nn.functional.relu(self.net[4](scores))
        scores = self.net[5](scores)
        scores = F.log_softmax(scores, dim=1)
        return scores


retrieval_model = feature_extractor()
retrieval_model.eval()
print("Model loaded")

test_loss, test_acc, _ = evaluate(mmodel=retrieval_model, current_graphs=graphs, batch_size=batch_size, num_classes=num_classes, device=device, out_dir=out_dir, last_round=True)
print("Evaluate: loss on test: ", test_loss, " and accuracy: ", test_acc * 100)

Creating model
Children Counter:  0  Layer Name:  convolution_1
Children Counter:  1  Layer Name:  convolution_2
Children Counter:  2  Layer Name:  convolution_3
Children Counter:  3  Layer Name:  attention
Children Counter:  4  Layer Name:  fully_connected_first
Children Counter:  5  Layer Name:  scoring_layer
Model loaded
Memory cleared. Current usage: 43.2 %


d:\3DStepGraphClassification_RapidDesignData\Graph_classification\train_utils.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  node_features_list = [torch.tensor(graph.node_features, dtype=torch.float).to(device) for graph in batch_graph]
d:\3DStepGraphClassification_RapidDesignData\Graph_classification\train_utils.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_indices = torch.tensor(graph.edge_mat, dtype=torch.long).to(device) + cumsum_nodes[i]


torch.Size([2, 5182])
test_adj: torch.Size([2, 5182])
prediction_scores: torch.Size([1, 393])
Memory cleared. Current usage: 44.7 %
torch.Size([2, 3226])
test_adj: torch.Size([2, 3226])
prediction_scores: torch.Size([1, 393])
Memory cleared. Current usage: 44.7 %
torch.Size([2, 1184])
test_adj: torch.Size([2, 1184])
prediction_scores: torch.Size([1, 393])
Memory cleared. Current usage: 44.7 %
torch.Size([2, 10860])
test_adj: torch.Size([2, 10860])
prediction_scores: torch.Size([1, 393])
Memory cleared. Current usage: 44.7 %
torch.Size([2, 13474])
test_adj: torch.Size([2, 13474])
prediction_scores: torch.Size([1, 393])
Memory cleared. Current usage: 44.7 %
torch.Size([2, 6720])
test_adj: torch.Size([2, 6720])
prediction_scores: torch.Size([1, 393])
Memory cleared. Current usage: 44.7 %
torch.Size([2, 3750])
test_adj: torch.Size([2, 3750])
prediction_scores: torch.Size([1, 393])
Memory cleared. Current usage: 44.7 %
torch.Size([2, 10860])
test_adj: torch.Size([2, 10860])
prediction_score

d:\3DStepGraphClassification_RapidDesignData\Graph_classification\train_utils.py:344: RuntimeWarning: invalid value encountered in divide
  matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]


FileNotFoundError: [Errno 2] No such file or directory: 'D:/3DStepGraphClassification/Output/results.txt'

In [1]:
#  old code
print("Creating model")

model = GCN_CN_v4(feature_dim_size=feature_dim_size, num_classes=num_classes, dropout=dropout).to(device)
model.load_state_dict(torch.load(model_path))
children_counter = 0
for n,c in model.named_children():
    print("Children Counter: ",children_counter," Layer Name: ",n,)
    children_counter+=1
output_layer = "attention"

class feature_extractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained = model
        self.pretrained.eval()

        self.net = list(self.pretrained.children())[:]#-2
        self.pretrained = None

    def forward(self, adj, features):
        features = self.net[0](x=features, edge_index=adj)
        features = nn.functional.relu(features)
        features = self.net[1](x=features, edge_index=adj)
        features = nn.functional.relu(features)
        features = self.net[2](x=features, edge_index=adj)
        features = nn.functional.relu(features)
        scores = self.net[3](features)
        scores = torch.t(scores)

        scores = nn.functional.relu(self.net[4](scores))
        scores = self.net[5](scores)
        scores = F.log_softmax(scores, dim=1)
        return scores

retrieval_model = feature_extractor()

test_loss, test_acc, _ = evaluate(mmodel=retrieval_model, current_graphs=graphs, batch_size=batch_size, num_classes=num_classes, device=device, out_dir=out_dir, last_round=True)
print("Evaluate: loss on test: ", test_loss, " and accuracy: ", test_acc * 100)

Creating model


NameError: name 'GCN_CN_v5' is not defined